# imports

In [82]:
# import matplotlib
# matplotlib.use("GTK4Agg")
# import matplotlib.pyplot as plt

# import matplotlib.rcsetup as rcsetup
# print(rcsetup.all_backends)
# print(matplotlib.get_backend())
# plt.plot(range(10))
# plt.show()

In [1]:
import sounddevice as sd
import librosa
import librosa.display
import numpy as np
import matplotlib
matplotlib.use("agg")
import matplotlib.pyplot as plt
print(matplotlib.get_backend())
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

agg


2023-08-20 16:38:09.956914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 16:38:10.782419: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# record and play

In [2]:
def record_audio(duration, sample_rate=22050):
    print("Recording audio...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
    sd.wait()  # Wait for recording to finish
    print("Recording finished.")

    # Convert the audio data to a numpy array
    audio_data_np = audio_data.squeeze()
    return audio_data, audio_data_np

def play_audio(audio_data, sample_rate=22050):
    print("Playing audio...")
    sd.play(audio_data, samplerate=sample_rate)
    sd.wait()  # Wait for playback to finish
    print("Playback finished.")

# signal augmentation

In [3]:
def augment_audio(audio_data, window_duration=3, overlap=1.5, desired_sr=22050):

    # Calculate the number of samples for each window
    window_size = int(window_duration * desired_sr)

    # Calculate the shift size of windows
    shift_size = int(desired_sr * (window_duration - overlap))

    # Split the audio into windows with overlap
    windows = []
    start = 0
    while start + window_size <= len(audio_data):
        windows.append(audio_data[start:start + window_size])
        start += shift_size

    # Convert the list of windows to a numpy array
    windows = np.array(windows)

    return windows

# preprocessing on input data

### converting numpy to MFCC

In [4]:
def signal2MFCC(windows, desired_sr=22050):
    
    windows = windows.astype(np.float32)
    mfcc_features = []
    
    for window in windows:
        mfcc = librosa.feature.mfcc(y=window, sr=desired_sr, n_mfcc=13)
        mfcc_features.append(mfcc)

    mfcc_features = np.array(mfcc_features)
    return mfcc_features

### reshaping input

In [5]:
def reshape_input(X_test_mfccs):
  X_test_mfcc_reshaped = X_test_mfccs.reshape(X_test_mfccs.shape[0], X_test_mfccs.shape[1], X_test_mfccs.shape[2], 1)

  # print(X_test_mfccs.shape , "reshaped: ", X_test_mfcc_reshaped.shape)
  return  X_test_mfcc_reshaped

### doing all preprocess phase

In [6]:
def preprocessing_input(audio_data_np, window_duration=3, overlap=1.5, desired_sr=22050):
    windows = augment_audio(audio_data_np, window_duration=window_duration, \
                            overlap=overlap, desired_sr=desired_sr)
    mfcc_windows = signal2MFCC(windows, desired_sr=desired_sr)
    mfcc_windows_reshaped = reshape_input(mfcc_windows)
    return mfcc_windows_reshaped

# model

### predidting

In [7]:
def test_model(model, mfccs, class_labels, do_print=False):
    # Predict the genre
    predictions = model.predict(mfccs)

    # Get the predicted genre label
    genre_label = np.argmax(predictions, axis=1)

    # Convert genre label to the actual genre name (assuming you have a list of genre names)
    predicted_genre = [class_labels[index] for index in genre_label]

    if do_print : print(predicted_genre)
    return predicted_genre

# calling methods 

In [8]:
# Set the duration and sample rate for recording
duration = 10  # Duration in seconds
window_duration=3
overlap=1.5 
desired_sr=22050
class_labels = ['A-Abuata',         #0
                'A-Afshari',        #1
                'A-Bayat-E Kord',   #2
                'A-Bayat-E Tork',   #3
                'A-Bayat-e Esfahan',#4
                'A-Dashti',         #5
                'D-Chahargah',      #6
                'D-Homayun',        #7
                'D-Mahoor',         #8
                'D-Nava',           #9
                'D-Rastpanjgah',    #10
                'D-Segah',          #11
                'D-Shur'            #12
                ]   

# Record audio
input_label = 0
audio_data, audio_data_np = record_audio(duration)

# Play the recorded audio
# play_audio(audio_data)

final_data = preprocessing_input(audio_data_np, window_duration=window_duration\
                                 ,overlap=overlap, desired_sr=desired_sr)

# y_train = np.full((final_data.shape)[0], input_label)
# num_classes = 13  # Number of classes
# y_train_1hot = to_categorical(y_train, num_classes=num_classes)

model = load_model('model-1.h5')

predicted_genre = test_model(model, final_data, class_labels, do_print=True)

Recording audio...
Recording finished.


2023-08-20 16:41:12.453963: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


1/1 [==============================] - 0s 189ms/step
['D-Shur', 'D-Shur', 'D-Shur', 'D-Shur', 'D-Shur']
